In [1]:
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import math
import time

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
count = 0
alldata = []
fps_time = 0

In [3]:
pose_tubuh = ['NOSE', 'LEFT_EYE_INNER', 'LEFT_EYE', 'LEFT_EYE_OUTER', 'RIGHT_EYE_INNER', 'RIGHT_EYE', 'RIGHT_EYE_OUTER', 'LEFT_EAR', 'RIGHT_EAR', 'MOUTH_LEFT', 'MOUTH_RIGHT',
              'LEFT_SHOULDER', 'RIGHT_SHOULDER', 'LEFT_ELBOW', 'RIGHT_ELBOW', 'LEFT_WRIST', 'RIGHT_WRIST', 'LEFT_PINKY', 'RIGHT_PINKY', 'LEFT_INDEX', 'RIGHT_INDEX', 'LEFT_THUMB',
              'RIGHT_THUMB', 'LEFT_HIP', 'RIGHT_HIP', 'LEFT_KNEE', 'RIGHT_KNEE', 'LEFT_ANKLE', 'RIGHT_ANKLE', 'LEFT_HEEL', 'RIGHT_HEEL', 'LEFT_FOOT_INDEX', 'RIGHT_FOOT_INDEX']


In [4]:
len(pose_tubuh)

33

In [5]:
cap = cv2.VideoCapture(0)

In [6]:
with mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image)

    # Draw the pose annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
    
    
    if results.pose_landmarks:
            data_tubuh = {}
            for i in range(len(pose_tubuh)):
                results.pose_landmarks.landmark[i].x = results.pose_landmarks.landmark[i].x * image.shape[0]
                results.pose_landmarks.landmark[i].y = results.pose_landmarks.landmark[i].y * image.shape[1]
                data_tubuh.update(
                {pose_tubuh[i] : results.pose_landmarks.landmark[i]}
                )
            alldata.append(data_tubuh)
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
    key=cv2.waitKey(1)
    if key==27:
        df = pd.DataFrame(alldata)
        df.to_csv("koordinat.csv")
        break
cap.release()
cv2.destroyAllWindows()


Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.


KeyboardInterrupt: 

In [7]:
df['NOSE'][1].x

NameError: name 'df' is not defined

In [8]:
df.columns

NameError: name 'df' is not defined

In [9]:
data=pd.DataFrame()

In [10]:
data

""


In [11]:
for name in pose_tubuh:
    for number in range(len(df[name])):
        namx=name+'x'
        data[namx]=df[name][number].x
        namy=name+'y'
        data[namy]=df[name][number].y
        namz=name+'z'
        data[namz]=df[name][number].z
        
        

NameError: name 'df' is not defined

In [12]:
len(df['NOSE'])

NameError: name 'df' is not defined

In [3]:
def calculateAngle(landmark1, landmark2, landmark3):
    x1, y1, _ = landmark1
    x2, y2, _ = landmark2
    x3, y3, _ = landmark3
    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))
    if angle < 0:
        angle += 360
    return angle

In [4]:
def classifyPose(landmarks, output_image, display=False):
   
    label = 'Unknown Pose'
    color = (0, 0, 255)
    
    # Calculate the required angles.
    #----------------------------------------------------------------------------------------------------------------
    
    # Get the angle between the left shoulder, elbow and wrist points. 
    left_elbow_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                      landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                      landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value])
    
    # Get the angle between the right shoulder, elbow and wrist points. 
    right_elbow_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value])
    # Get the angle between the right shoulder, hip and knee points. 
    right_hip_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value])
    # Get the angle between the left shoulder, hip and knee points. 
    left_hip_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                       landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                       landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value])
    
    # Get the angle between the left elbow, shoulder and hip points. 
    left_shoulder_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                         landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                         landmarks[mp_pose.PoseLandmark.LEFT_HIP.value])

    # Get the angle between the right hip, shoulder and elbow points. 
    right_shoulder_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                          landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                          landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value])

    # Get the angle between the left hip, knee and ankle points. 
    left_knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                     landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value],
                                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value])

    # Get the angle between the right hip, knee and ankle points 
    right_knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                      landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
                                      landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value])
    
    
    middle_angle=calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
                                      ((landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value][0]+landmarks[mp_pose.PoseLandmark.LEFT_HIP.value][0])/2,(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value][1]+landmarks[mp_pose.PoseLandmark.LEFT_HIP.value][1])/2,(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value][2]+landmarks[mp_pose.PoseLandmark.LEFT_HIP.value][2])),
                                      landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value])
    
    middle_angle = 360-middle_angle
    #----------------------------------------------------------------------------------------------------------------
    
    # Check if it is the warrior II pose or the T pose.
    # As for both of them, both arms should be straight and shoulders should be at the specific angle.
    #----------------------------------------------------------------------------------------------------------------
    
    # Check if the both arms are straight.
    if left_elbow_angle > 165 and left_elbow_angle < 195 and right_elbow_angle > 165 and right_elbow_angle < 195:

        # Check if shoulders are at the required angle.
        if left_shoulder_angle > 80 and left_shoulder_angle < 110 and right_shoulder_angle > 80 and right_shoulder_angle < 110:

# Check if it is the warrior II pose.
    #----------------------------------------------------------------------------------------------------------------

            # Check if one leg is straight.
            if left_knee_angle > 165 and left_knee_angle < 195 or right_knee_angle > 165 and right_knee_angle < 195:

                # Check if the other leg is bended at the required angle.
                if left_knee_angle > 80 and left_knee_angle < 120 or right_knee_angle > 80 and right_knee_angle < 120:

                    # Specify the label of the pose that is Warrior II pose.
                    label = 'Warrior II Pose' 
                        
    #----------------------------------------------------------------------------------------------------------------
    
    # Check if it is the T pose.
    #----------------------------------------------------------------------------------------------------------------
    
            # Check if both legs are straight
            if left_knee_angle > 160 and left_knee_angle < 195 and right_knee_angle > 160 and right_knee_angle < 195:

                # Specify the label of the pose that is tree pose.
                label = 'T Pose'

    #----------------------------------------------------------------------------------------------------------------
    
    # Check if it is the tree pose.
    #----------------------------------------------------------------------------------------------------------------
    
    # Check if one leg is straight
    if left_knee_angle > 165 and left_knee_angle < 195 or right_knee_angle > 165 and right_knee_angle < 195:

        # Check if the other leg is bended at the required angle.
        if left_knee_angle > 315 and left_knee_angle < 335 or right_knee_angle > 25 and right_knee_angle < 45:

            # Specify the label of the pose that is tree pose.
            label = 'Tree Pose'
                
    #----------------------------------------------------------------------------------------------------------------
#____________________________________________________________________________________________________________________
        
   
#     # Check if it is the Trikonasana pose.
    #----------------------------------------------------------------------------------------------------------------
    
    if(middle_angle<85 and middle_angle>75):
        if((right_shoulder_angle>118 and right_shoulder_angle<130) and (left_shoulder_angle>85 and left_shoulder_angle<95)) or ((left_shoulder_angle>118 and left_shoulder_angle<130) and (right_shoulder_angle>85 and right_shoulder_angle<95)):
            label='Trikonasana'
     #----------------------------------------------------------------------------------------------------------------
    
    # Check if it is the Ardha chandrasana pose.
    #----------------------------------------------------------------------------------------------------------------
    
    if(middle_angle>95 and middle_angle<105):
        if((right_shoulder_angle>90 and right_shoulder_angle<100) and (left_shoulder_angle>95 and left_shoulder_angle<105)) or ((left_shoulder_angle>90 and left_shoulder_angle<100) and (right_shoulder_angle>95 and right_shoulder_angle<105)):
            if((right_hip_angle>170 and right_hip_angle<190) or (left_hip_angle>170 and left_hip_angle<190)):
                label='Ardha chandrasana'
     #----------------------------------------------------------------------------------------------------------------
    
    # Check if it is the Parsvakonasana pose.
    #----------------------------------------------------------------------------------------------------------------
    if(middle_angle>130 and middle_angle<140):
        if((right_shoulder_angle>175 and right_shoulder_angle<195) and (left_shoulder_angle>40 and left_shoulder_angle<55)) or ((left_shoulder_angle>175 and left_shoulder_angle<195) and (right_shoulder_angle>40 and right_shoulder_angle<55)):
            if(left_knee_angle>240 and left_knee_angle<255) or (right_knee_angle>110 and right_knee_angle<120):
                label='Parsvakonasana'
     #----------------------------------------------------------------------------------------------------------------
    label=label
    
    
    
    
    
    
    
    
    
    #____________________________________________________________________________________________________________________
    
    if label != 'Unknown Pose':
         color = (0, 255, 0)  
    cv2.putText(output_image, label, (10, 30),cv2.FONT_HERSHEY_PLAIN, 2, color, 2) 
    if display:
        plt.figure(figsize=[10,10])
        plt.imshow(output_image[:,:,::-1]);plt.title("Output Image");plt.axis('off');
        
    else:        
        return output_image, label

In [5]:
def detectPose(image, pose, display=True):
    output_image = image.copy()
    imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(imageRGB)
    height, width, _ = image.shape
    landmarks = []
    
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image=output_image, landmark_list=results.pose_landmarks,
                                  connections=mp_pose.POSE_CONNECTIONS)        
        for landmark in results.pose_landmarks.landmark:           
            landmarks.append((int(landmark.x * width), int(landmark.y * height),
                                  (landmark.z * width)))
    
    if display:    
        plt.figure(figsize=[22,22])
        plt.subplot(121);plt.imshow(image[:,:,::-1]);plt.title("Original Image");plt.axis('off');
        plt.subplot(122);plt.imshow(output_image[:,:,::-1]);plt.title("Output Image");plt.axis('off');        
        mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
    else:        
        return output_image, landmarks

In [6]:
pose_video = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, model_complexity=1)
camera_video = cv2.VideoCapture(0)
# camera_video.set(3,1280)
# camera_video.set(4,960)
cv2.namedWindow('Pose Classification', cv2.WINDOW_NORMAL)
while camera_video.isOpened():    
    ok, frame = camera_video.read() 
    if not ok:
        continue    
    frame = cv2.flip(frame, 1)
    frame_height, frame_width, _ =  frame.shape
    frame = cv2.resize(frame, (int(frame_width * (640 / frame_height)), 640))
    frame, landmarks = detectPose(frame, pose_video, display=False)
    if landmarks:
        frame, _ = classifyPose(landmarks, frame, display=False)
    cv2.imshow('Pose Classification', frame)
    k = cv2.waitKey(1) & 0xFF
    if(k == 27):
        break
camera_video.release()
cv2.destroyAllWindows()